In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt

In [3]:
CLIP_MIN = 0.0
CLIP_MAX = 1.0

# CLIP_MIN = -1
# CLIP_MAX = 2.0

In [4]:
def load_data(run='1', net='cif10/wrn_28_10_10'):
    my_path      = '/home/lorenzp/adversialml/src'

    originals    = torch.load(my_path + '/data/attacks/run_{}/'.format(run) + net + '/bim/images')

    gauss        = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/gauss/images_adv')
    fgsms        = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/fgsm/images_adv')
    bims         = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/bim/images_adv')
    pgds         = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/pgd/images_adv')
    aa           = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/std/8_255/images_adv')
    deepfools    = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/df/images_adv')
    cws          = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/cw/images_adv')

    o_gauss      = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/gauss/images')
    o_fgsms      = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/fgsm/images')
    o_bims       = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/bim/images')
    o_pgds       = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/pgd/images')
    o_aa         = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/std/8_255/images')
    o_deepfools  = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/df/images')
    o_cws        = torch.load(my_path +  '/data/attacks/run_{}/'.format(run) + net + '/cw/images')

    tmp_attacks = [originals, gauss,       fgsms,   bims,   pgds,   aa,   deepfools,  cws]
    tmp_origs   = [originals, o_gauss,   o_fgsms, o_bims, o_pgds, o_aa, o_deepfools, o_cws]
    tmp_attack_name = ['normal', 'Gaussian', 'FGSM', 'BIM', ' PGD', 'AA',  'Deepfool', 'C&W']

    return tmp_attacks, tmp_origs, tmp_attack_name

In [5]:
attacks, origs, attack_name = load_data(run=1, net='cif10/wrn_28_10_10')       

In [6]:
def get_noisy_samples(X_test, dataset='cif10', stddev=8/255):
    # print("stddev", stddev)
    X_test_noisy = np.minimum(
        np.maximum(
            X_test + np.random.normal(loc=0, scale=stddev, size=X_test.shape).astype('f'),
            CLIP_MIN
        ),
        CLIP_MAX
    )

    return X_test_noisy

In [7]:
def find_dev(attacks, origs, attack_name, stddeviations):
    
    for att in range(len(attacks)):
        print("================================================")
        print("attack: ", attack_name[att])
        min_dist_err = 100000000
        min_stddev = 0
        per_sample_min_stddev = [] 

        i_att   = attacks[att]
        i_origs = origs[att]
        
        origs_stddev = []
        for i in range(20):
            i_att_tmp = i_att[i].cpu()
            i_origs_tmp = i_origs[i].cpu()
            
            o_dist_err = torch.dist( i_origs_tmp, i_att_tmp , p=2 )
            # o_dist_err = np.linalg.norm(i_origs_tmp - i_att_tmp, 1)
            
            origs_stddev.append(o_dist_err)
            print("orig:     ", o_dist_err)
            for it_stddev in stddeviations:
                
                noisy = torch.tensor( get_noisy_samples( i_origs_tmp.numpy(), stddev=it_stddev) )
                dist_err = torch.dist( i_origs_tmp, noisy , p=2)
                # dist_err = np.linalg.norm( i_origs_tmp - noisy )
                
                print("noisy     ", dist_err)
                
                # if np.abs( dist_err - o_dist_err ) <= min_dist_err:
                #     print("noisy", dist_err)
                #     per_sample_min_stddev.append( stddev )
        
        print("average l2 dist: ", np.mean(origs_stddev))
        
        # print("per_sample_min_stddev: ", per_sample_min_stddev)
        
        

        print("--------------")


In [8]:
# std_devs = [  0.011/255., 0.1/255., 0.2/255., 0.5 / 255., 1 / 255., 2 / 255., 3 / 255., 4 / 255., 8 / 255.]
# std_devs = [  0.001/255., 0.01/255.,  0.1/255.,  0.2/255.]
# std_devs = [  0.000001, 0.00001, 0.0001, 0.001]
# std_devs = [ 0.015686, 0.0015686, 0.15686 ]

# STD = np.linspace(0.0001, 0.1, 1000)
# STD = [0.002080]

STD = [0.1172]

In [9]:
attack_name

['normal', 'Gaussian', 'FGSM', 'BIM', ' PGD', 'AA', 'Deepfool', 'C&W']

In [10]:
print(attack_name[2])

i_att   = attacks[2]
i_origs = origs[2]

i_att_tmp = i_att[0].cpu()
i_origs_tmp = i_origs[0].cpu()

i_origs_tmp.shape

FGSM


torch.Size([3, 32, 32])

In [11]:
o_dist_err = torch.dist( i_origs_tmp, i_att_tmp , p=2 )
o_dist_err

tensor(1.7357)

In [12]:
noisy = torch.tensor( get_noisy_samples( i_origs_tmp.numpy(), stddev=0.02915877943) )
dist_err = torch.dist( i_origs_tmp, noisy , p=2)
dist_err

tensor(1.6247)

In [13]:
iter_sample = 1

In [22]:
NR_ATT = 7


print(attack_name[NR_ATT])

i_att   = attacks[NR_ATT]
i_origs = origs[NR_ATT]

i_att_tmp = i_att[0].cpu()
i_origs_tmp = i_origs[0].cpu()


lp=1
s = (32**2 *3)

std_deviations = []

for iter_sample in range(1000):
    i_att_tmp = i_att[iter_sample].cpu()
    i_origs_tmp = i_origs[iter_sample].cpu()


    o_dist_err = torch.dist( i_origs_tmp, i_att_tmp , p=lp )
    # print("o_dist_err: ", o_dist_err)

    if lp == 1:
        std = o_dist_err /  s * np.sqrt(np.pi/2)
    elif lp == 2:
        std = np.sqrt(o_dist_err /  s) * np.sqrt(np.pi/2)
        
    std_deviations.append(std)
    # print("standard: ", std)

    noisy = torch.tensor( get_noisy_samples( i_origs_tmp.numpy(), stddev=std) )
    dist_err = torch.dist( i_origs_tmp, noisy , p=lp)

    # print("dist_err:   ", dist_err)

print("mean std: ", np.mean(std_deviations))

C&W
mean std:  0.0020574536


In [ ]:
    for att in range(len(attacks)):
        print("================================================")
        print("attack: ", attack_name[att])
        min_dist_err = 100000000
        min_stddev = 0
        per_sample_min_stddev = [] 

        i_att   = attacks[att]
        i_origs = origs[att]
        
        origs_stddev = []
        for i in range(20):
            i_att_tmp = i_att[i].cpu()
            i_origs_tmp = i_origs[i].cpu()
            
            o_dist_err = torch.dist( i_origs_tmp, i_att_tmp , p=2 )
            # o_dist_err = np.linalg.norm(i_origs_tmp - i_att_tmp, 1)
            
            origs_stddev.append(o_dist_err)
            print("orig:     ", o_dist_err)
            for it_stddev in stddeviations:
                
                noisy = torch.tensor( get_noisy_samples( i_origs_tmp.numpy(), stddev=it_stddev) )
                dist_err = torch.dist( i_origs_tmp, noisy , p=2)
                # dist_err = np.linalg.norm( i_origs_tmp - noisy )
                
                print("noisy     ", dist_err)
        
        print("average l2 dist: ", np.mean(origs_stddev))
        

In [168]:
# it = 2
# find_dev([attacks[it]], origs, [attack_name[it]], STD)


find_dev(attacks, origs, attack_name, STD)

attack:  normal
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.3638)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(5.8498)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.3157)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.5745)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.3944)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.3891)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(5.9490)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.3318)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.4325)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.2678)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.3708)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.0845)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.2360)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(5.8538)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.3708)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(

In [169]:
attacks, origs, attack_name = load_data(run=1, net='cif100/wrn_28_10_100')
find_dev(attacks, origs, attack_name, STD)

attack:  normal
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.4170)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.4062)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.2743)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.2601)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.3997)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.2730)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.3899)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.2828)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.1806)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.0798)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(5.3325)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.4760)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(5.8308)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.2249)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(6.0608)
orig:      tensor(0.)
stddev 0.1172
noisy      tensor(

In [170]:
attacks, origs, attack_name = load_data(run=1, net='cif10vgg/vgg_16_0_10')
find_dev(attacks, origs, attack_name)

TypeError: find_dev() missing 1 required positional argument: 'stddeviations'

In [ ]:
attacks, origs, attack_name = load_data(run=1, net='cif100vgg/vgg_16_0_100')
find_dev(attacks, origs, attack_name, STD)

In [ ]:
attacks, origs, attack_name = load_data(run=1, net='imagenet/wrn_50_2')
find_dev(attacks, origs, attack_name, STD)